Lokasi pengantaran  
0 : Lokasi A  
1 : Lokasi B  
2 : Lokasi C  

Lokasi asal dan kembali  
3 : Lokasi X  
4 : Lokasi Y

In [52]:
import itertools
perm = [l for l in itertools.permutations(range(3), 3)]
print(perm)

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]


In [53]:
lokasi_asal = [3, 4]

In [54]:
distances = {
    (0,0): 0,     (0,1): 10694, (0,2): 11951, (0,3): 12641, (0,4): 13651,
    (1,0): 10694, (1,1): 0,     (1,2): 12251, (1,3): 13001, (1,4): 14051,
    (2,0): 11951, (2,1): 12251, (2,2): 0,     (2,3): 12691, (2,4): 13341,
    (3,0): 12641, (3,1): 13001, (3,2): 12691, (3,3): 0,     (3,4): 12691,
    (4,0): 13651, (4,1): 14051, (4,2): 13341, (4,3): 12691, (4,4): 0,
}

In [73]:
paths = []
path_distances = {}

for i in range(len(lokasi_asal)):
    for j in range(len(perm)):
        path = (lokasi_asal[i],) + perm[j] + (lokasi_asal[i],)
        paths.append(path)
        path_distances[path] = 0
        
        for k in range(len(path)-1):
            edge = (path[k], path[k+1])
            path_distances[path] += distances[edge]

In [74]:
paths

[(3, 0, 1, 2, 3),
 (3, 0, 2, 1, 3),
 (3, 1, 0, 2, 3),
 (3, 1, 2, 0, 3),
 (3, 2, 0, 1, 3),
 (3, 2, 1, 0, 3),
 (4, 0, 1, 2, 4),
 (4, 0, 2, 1, 4),
 (4, 1, 0, 2, 4),
 (4, 1, 2, 0, 4),
 (4, 2, 0, 1, 4),
 (4, 2, 1, 0, 4)]

In [75]:
path_distances

{(3, 0, 1, 2, 3): 48277,
 (3, 0, 2, 1, 3): 49844,
 (3, 1, 0, 2, 3): 48337,
 (3, 1, 2, 0, 3): 49844,
 (3, 2, 0, 1, 3): 48337,
 (3, 2, 1, 0, 3): 48277,
 (4, 0, 1, 2, 4): 49937,
 (4, 0, 2, 1, 4): 51904,
 (4, 1, 0, 2, 4): 50037,
 (4, 1, 2, 0, 4): 51904,
 (4, 2, 0, 1, 4): 50037,
 (4, 2, 1, 0, 4): 49937}